In [1]:
import arcpy, os
import pandas as pd
from arcpy import env
from arcpy.ia import *
env.overwriteOutput = True
from arcgis.features import GeoSeriesAccessor,GeoAccessor
from arcgis import features

In [2]:
path = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\KML'
arcpy.env.workspace = path

shp = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP'

In [3]:
# Lista todos os arquivos KML na pasta
kml_files = [f for f in os.listdir(path) if f.lower().endswith('.kml')]

In [4]:
# Loop para converter cada KML em camada
for kml in kml_files:
    kml_path = os.path.join(path, kml)
    nome_base = os.path.splitext(kml)[0]  # Remove a extensão .kml para usar como nome da camada
    arcpy.conversion.KMLToLayer(kml_path, shp, nome_base, "NO_GROUNDOVERLAY", "")

print("Conversão concluída para todos os arquivos KML.")

Conversão concluída para todos os arquivos KML.


In [5]:
#MERGING

In [6]:
# path2 = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP'
# arcpy.env.workspace = path2

In [7]:
# Define o shapefile de saída para o merge
# output_merge = os.path.join(path2, "merged_output.shp")
#output_merge = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP\merged_output.shp'

In [8]:
# 1. Defina o caminho da pasta onde estão os GDBs
pasta_gdbs = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP'

# 2. Defina o caminho de saída para o arquivo merge
output_gdb = r'C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\Results\output.gdb'
nome_feicao_saida = "Polylines_Merge"

# 3. Verifica se o GDB de saída existe; se não, cria ele
if not arcpy.Exists(output_gdb):
    print(f"Criando GDB de saída em: {output_gdb}")
    arcpy.management.CreateFileGDB(os.path.dirname(output_gdb), os.path.basename(output_gdb))
else:
    print(f"GDB de saída já existe: {output_gdb}")

# 4. Lista todos os GDBs na pasta
gdbs = [os.path.join(pasta_gdbs, gdb) for gdb in os.listdir(pasta_gdbs) if gdb.endswith(".gdb")]

# 5. Lista para armazenar todas as feições do tipo Polyline
feicoes_para_merge = []

# 6. Loop através de cada GDB
for gdb in gdbs:
    arcpy.env.workspace = gdb
    print(f"\nProcessando GDB: {gdb}")

    # Verifica datasets dentro do GDB
    datasets = arcpy.ListDatasets()
    for dataset in datasets:
        print(f"  Verificando dataset: {dataset}")
        arcpy.env.workspace = os.path.join(gdb, dataset)
        # Lista todas as feições, independentemente do tipo
        feicoes = arcpy.ListFeatureClasses()
        print(f"    Feições encontradas: {feicoes}")
        # Filtra apenas feições do tipo Polyline
        for feicao in feicoes:
            desc = arcpy.Describe(feicao)
            if desc.shapeType == "Polyline":
                feicoes_para_merge.append(os.path.join(gdb, dataset, feicao))
                print(f"      Feição adicionada: {feicao} (Tipo: {desc.shapeType})")

    # Verifica também na raiz do GDB
    arcpy.env.workspace = gdb
    feicoes = arcpy.ListFeatureClasses()
    print(f"  Feições na raiz do GDB: {feicoes}")
    for feicao in feicoes:
        desc = arcpy.Describe(feicao)
        if desc.shapeType == "Polyline":
            feicoes_para_merge.append(os.path.join(gdb, feicao))
            print(f"    Feição adicionada: {feicao} (Tipo: {desc.shapeType})")

# 7. Verifica se há feições para merge
if len(feicoes_para_merge) > 0:
    # Realiza o merge
    print(f"\nRealizando merge de {len(feicoes_para_merge)} feições...")
    arcpy.management.Merge(feicoes_para_merge, os.path.join(output_gdb, nome_feicao_saida))
    print(f"Merge concluído! Feições salvas em {os.path.join(output_gdb, nome_feicao_saida)}")
else:
    print("\nNenhuma feição do tipo Polyline encontrada para merge.")

Criando GDB de saída em: C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\Results\output.gdb

Processando GDB: C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP\01_1.gdb
  Verificando dataset: Placemarks
    Feições encontradas: ['Polylines']
      Feição adicionada: Polylines (Tipo: Polyline)
  Feições na raiz do GDB: []

Processando GDB: C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP\01_2.gdb
  Verificando dataset: Placemarks
    Feições encontradas: ['Polylines']
      Feição adicionada: Polylines (Tipo: Polyline)
  Feições na raiz do GDB: []

Processando GDB: C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP\01_3.gdb
  Verificando dataset: Placemarks
    Feições encontradas: ['Polylines']
      Feição adicionada: Polylines (Tipo: Polyline)
  Feições na raiz do GDB: []

Processando GDB: C:\Users\silvio.lemos\D\SL_GIS_PROJECTS\03_KML_TO_SHP\SHP\01_4.gdb
  Verificando dataset: Placemarks
    Feições encontradas: ['Polylines']
      Feição adicionada: Po

In [9]:
#CRIANDO CAMPO DE DISTANCIA

In [10]:
arcpy.management.AddField("Polylines_Merge","dist","DOUBLE")

<Result 'Polylines_Merge'>

In [13]:
spatial_reference = arcpy.SpatialReference(4326)

In [14]:
arcpy.management.CalculateGeometryAttributes("Polylines_Merge","dist LENGTH_GEODESIC","METERS","",spatial_reference,"SAME_AS_INPUT")

<Result 'Polylines_Merge'>